In [1]:
%autosave 600
import numpy as np
import pandas as pd
import random
import cupy
import spacy
import chainer
from spacy.util import minibatch, compounding
from sklearn.metrics import roc_auc_score

import thinc.neural.gpu_ops

Autosaving every 600 seconds


In [18]:
# load data
train = pd.read_csv('../input/train.csv')
trainX = train['comment_text'].values.tolist()

# set variables
n_texts = 10
tv_split = 0.8
n_iter = 8
class_names = list(train)[-6:]
# tox_type = 'threat'



In [19]:

def create_model(tox_type):



def preprocess(tox_type):
  train[tox_type] = pd.to_numeric(train[tox_type])
  target = train[tox_type].values.tolist()

  train_data = list(zip(trainX, target))
  random.shuffle(train_data)
  train_data = train_data[-n_texts:]

  texts, labels = zip(*train_data)
  cats = [{tox_type: bool(y)} for y in labels]
  split = int(len(train_data) * tv_split)

  train_texts = texts[:split]
  train_cats = cats[:split]
  dev_texts = texts[split:]
  dev_cats = cats[split:]

  print("Using {} examples ({} training, {} evaluation)"
      .format(len(texts), len(train_texts), len(dev_texts)))
  train_data = list(zip(train_texts,
                      [{'cats': cats} for cats in train_cats]))
  return train_data, dev_texts, dev_cats


def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 1e-8  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 1e-8  # True negatives
    
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            print (score)
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)

    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score}

In [22]:
for cl in class_names:
train_data, dev_texts, dev_cats = preprocess(cl)
nlp, clf, optimizer = create_model(cl)

print("Training the model...")
print('{:^5}\t{:^5}\t{:^5}'.format('iter#', 'LOSS', 'ROC'))

for i in range(n_iter):
losses = {}

batches = minibatch(train_data, size=compounding(4., 16., 1.001))
for batch in batches:
    texts, annotations = zip(*batch)
    nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)

#     with clf.model.use_params(optimizer.averages):
#         score = roc_auc_score(dev_cats, annotations)

print(i, dev_cats)


Using 10 examples (8 training, 2 evaluation)
Training the model...
iter#	LOSS 	 ROC 
0 [{'toxic': False}, {'toxic': True}]
1 [{'toxic': False}, {'toxic': True}]
2 [{'toxic': False}, {'toxic': True}]
3 [{'toxic': False}, {'toxic': True}]
4 [{'toxic': False}, {'toxic': True}]
5 [{'toxic': False}, {'toxic': True}]
6 [{'toxic': False}, {'toxic': True}]
7 [{'toxic': False}, {'toxic': True}]
Using 10 examples (8 training, 2 evaluation)
Training the model...
iter#	LOSS 	 ROC 
0 [{'severe_toxic': False}, {'severe_toxic': False}]
1 [{'severe_toxic': False}, {'severe_toxic': False}]
2 [{'severe_toxic': False}, {'severe_toxic': False}]
3 [{'severe_toxic': False}, {'severe_toxic': False}]
4 [{'severe_toxic': False}, {'severe_toxic': False}]
5 [{'severe_toxic': False}, {'severe_toxic': False}]
6 [{'severe_toxic': False}, {'severe_toxic': False}]
7 [{'severe_toxic': False}, {'severe_toxic': False}]
Using 10 examples (8 training, 2 evaluation)
Training the model...
iter#	LOSS 	 ROC 
0 [{'obscene': Fa

In [23]:
# test the trained model
test_text = " i must kill you !"
doc = nlp(test_text)
print(test_text, doc.cats)


 i must kill you ! {'identity_hate': 0.01189790666103363}


In [11]:
losses['textcat']

0.034178385382801935